### Yolo Architecture (Architecture; 구성 요소, 동작 원리)

YOLOv5 논문 기반 모델 구현

In [ ]:
# 기본 임포트 및 설정
import torch
import torch.nn as nn
# PyTorch의 신경망 모듈(neural network module)을 가져와서 사용
# torch.nn은 신경망을 구성하기 위한 레이어, 손실 함수, 활성화 함수 등을 제공함
import torch.nn.functional as F

print("✅ PyTorch version:", torch.__version__)

✅ PyTorch version: 2.9.0+cpu


In [ ]:
# ConvBlock 정의 (YOLO 기본 구성 블록)
class ConvBlock(nn.Module):
    # ConvBlock은 PyTorch의 기본 신경망 모듈 클래스 nn.Module을 상속 받음
    # -> 이 클래스는 하나의 신경망 층 단위(모듈)로 작동할 수 있음
    def __init__(self, in_channels, out_channels, kernel_size, stride=1):
        super().__init__()
        # super().__init__()는 부모 클래스(nn.Module)의 초기화 함수를 실행하여 **필요한 내부 구조를 세팅함**
        padding = kernel_size // 2  # same padding 유지
        # padding : 이미지 사이즈 동일하게 하기 위한 작업
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.LeakyReLU(0.1, inplace=True)
        # Conv2D → BatchNorm → LeakyReLU
        # -> 기본 CNN 단위 / 많은 모델이 이 형태를 기본으로 사용함
    # 순전파
    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

# ✅ 테스트
x = torch.randn(1, 3, 416, 416)
# (배치 1개, 채널 3, 크기 416×416)짜리 임의의 입력 이미지 생성
block = ConvBlock(3, 32, 3)
print("출력 shape:", block(x).shape)

출력 shape: torch.Size([1, 32, 416, 416])


In [ ]:
# Backbone 정의 (CSPDarknet 기반 간소화 버전)
# (Feature Extractor)
class YOLOBackbone(nn.Module):
    # YOLOBackbone : nn.Module을 상속받는 PyTorch 신경망 클래스
    # -> 이 클래스는 하나의 신경망 층 단위(모듈)로 작동할 수 있음
    # (back bone) 입력 이미지의 주요 특징(feature map)을 추출하는 역할을 함
    def __init__(self):
        super().__init__()

        # ConvBlock(Conv2D + BatchNorm + LeakyReLU) 블록을 사용해서 점점 깊어지는 CNN 네트워크를 구성함
        # (입력 채널, 출력 채널, 커널 크기;3*3, stride)
        self.layer1 = ConvBlock(3, 32, 3, 1) # 처음 입력 이미지를 처리 (크기 유지)
        self.layer2 = ConvBlock(32, 64, 3, 2) # stride=2 : 해상도 절반으로 줄임
        self.layer3 = ConvBlock(64, 128, 3, 2)
        self.layer4 = ConvBlock(128, 256, 3, 2)
        self.layer5 = ConvBlock(256, 512, 3, 2)
        # 계층이 깊어질수록 채널 수는 증가하고, 공간 크기는 감소
        # 이건 CNN의 전형적인 구조이며, 점점 더 추상적이고 강력한 특징을 학습하게 됨

    # 순전파
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        # 입력 이미지 x를 순서대로 각 합성곱 블록에 통과시키며 계속 특징을 추출하고 크기를 줄임

        return x

# ✅ 테스트
backbone = YOLOBackbone()
# instance (객체화)
feat = backbone(torch.randn(1, 3, 416, 416))
print("백본 출력 shape:", feat.shape)
# feet.shape : 결과 tensor 의 크기 확인

# 1) stride 전체 = 1 * 2 * 2 * 2 * 2 = 16배 축소 (원본 이미지 총 16배 축소됨)
# 2) 입력 (416*416) -> 출력(416/16 = 26*26)
# 3) 출력 채널 512

# 채널 수 ↑, 공간 크기 ↓ → 깊은 특징 추출

백본 출력 shape: torch.Size([1, 512, 26, 26])


---

In [ ]:
# Detection Head (Bounding Box + Class Prediction)
# (수정: self.num_classes 사용)
class YOLOHead(nn.Module):
    def __init__(self, in_channels, num_classes, num_anchors=3):
        super().__init__()
        self.num_classes = num_classes   # ← self.num_classes 로 저장
        self.num_anchors = num_anchors
        self.pred = nn.Conv2d(in_channels, num_anchors * (5 + num_classes), 1)
        # (5: x, y, w, h, confidence)

    def forward(self, x):
        out = self.pred(x)
        B, _, H, W = out.shape
        out = out.view(B, self.num_anchors, 5 + self.num_classes, H, W)
        return out

# ✅ 테스트
head = YOLOHead(in_channels=512, num_classes=3)
out = head(torch.randn(1, 512, 13, 13))
print("헤드 출력 shape:", out.shape)

헤드 출력 shape: torch.Size([1, 3, 8, 13, 13])


In [13]:
# YOLOv5 전체 모델 조합 (Backbone + Head)
class YOLOv5(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.backbone = YOLOBackbone()
        self.head = YOLOHead(in_channels=512, num_classes=num_classes)

    def forward(self, x):
        feats = self.backbone(x)
        preds = self.head(feats)
        return preds

# ✅ 테스트
model = YOLOv5(num_classes=3)
preds = model(torch.randn(1, 3, 416, 416))
print("모델 출력 shape:", preds.shape)

모델 출력 shape: torch.Size([1, 3, 8, 26, 26])


In [14]:
# 손실 함수 (Eq.3 기반 YOLO Loss)
# (Localization + Confidence + Class)
class YOLOLoss(nn.Module):
    def __init__(self, lambda_coord=5.0, lambda_noobj=0.5):
        super().__init__()
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj

    def forward(self, predictions, targets):
        # predictions, targets: [B, A, 5+C, H, W]
        xy_loss = F.mse_loss(predictions[..., 0:2], targets[..., 0:2], reduction="sum")
        wh_loss = F.mse_loss(torch.sqrt(predictions[..., 2:4].clamp(min=1e-6)),
                             torch.sqrt(targets[..., 2:4].clamp(min=1e-6)), reduction="sum")
        conf_loss_obj = F.binary_cross_entropy_with_logits(
            predictions[..., 4], targets[..., 4], reduction="sum"
        )
        conf_loss_noobj = F.binary_cross_entropy_with_logits(
            predictions[..., 4], targets[..., 4], reduction="sum"
        )
        cls_loss = F.binary_cross_entropy_with_logits(
            predictions[..., 5:], targets[..., 5:], reduction="sum"
        )

        total_loss = (
            self.lambda_coord * (xy_loss + wh_loss)
            + conf_loss_obj
            + self.lambda_noobj * conf_loss_noobj
            + cls_loss
        )
        return total_loss

# ✅ 테스트
loss_fn = YOLOLoss()
pred = torch.randn(1, 3, 8, 13, 13)
target = torch.randn(1, 3, 8, 13, 13)
print("loss:", loss_fn(pred, target).item())

loss: 10700.7421875


In [ ]:
# IoU 계산 함수 (Eq.5 기반)
# (Intersection over Union)
def bbox_iou(box1, box2):
    # box = [x_center, y_center, w, h]
    b1_x1, b1_y1 = box1[0] - box1[2]/2, box1[1] - box1[3]/2
    b1_x2, b1_y2 = box1[0] + box1[2]/2, box1[1] + box1[3]/2
    b2_x1, b2_y1 = box2[0] - box2[2]/2, box2[1] - box2[3]/2
    b2_x2, b2_y2 = box2[0] + box2[2]/2, box2[1] + box2[3]/2

    inter_x1, inter_y1 = torch.max(b1_x1, b2_x1), torch.max(b1_y1, b2_y1)
    inter_x2, inter_y2 = torch.min(b1_x2, b2_x2), torch.min(b1_y2, b2_y2)
    inter_area = torch.clamp(inter_x2 - inter_x1, min=0) * torch.clamp(inter_y2 - inter_y1, min=0)

    union_area = (box1[2] * box1[3]) + (box2[2] * box2[3]) - inter_area
    iou = inter_area / (union_area + 1e-6)
    return iou

# ✅ 테스트
b1 = torch.tensor([50, 50, 40, 40])
b2 = torch.tensor([60, 60, 40, 40])
print("IoU:", bbox_iou(b1, b2).item())

IoU: 0.3913043439388275


In [16]:
# Non-Max Suppression (중복 박스 제거) (NMS)
def non_max_suppression(predictions, conf_thresh=0.5, iou_thresh=0.5):
    preds = predictions[predictions[:, 4] > conf_thresh]
    if preds.shape[0] == 0:
        return []
    preds = preds[(-preds[:, 4]).argsort()]  # confidence 내림차순 정렬
    keep = []
    while preds.size(0):
        top = preds[0]
        keep.append(top)
        if preds.size(0) == 1:
            break
        ious = torch.tensor([bbox_iou(top[:4], box[:4]) for box in preds[1:]])
        preds = preds[1:][ious < iou_thresh]
    return torch.stack(keep) if keep else torch.tensor([])

# ✅ 테스트
sample_preds = torch.tensor([
    [50, 50, 40, 40, 0.9, 0],
    [55, 55, 42, 42, 0.8, 0],
    [120, 120, 30, 30, 0.85, 1]
])
print("NMS 결과:", non_max_suppression(sample_preds))

NMS 결과: tensor([[ 50.0000,  50.0000,  40.0000,  40.0000,   0.9000,   0.0000],
        [120.0000, 120.0000,  30.0000,  30.0000,   0.8500,   1.0000]])


⚙️ 다음 단계 (선택지)

이제 선택할 수 있는 다음 단계는 두 가지예요 👇

🧠 훈련(Training) 루프 작성

→ optimizer, loss.backward(), optimizer.step() 포함해서 실제 학습을 돌리는 코드

🖼️ 예측 & 시각화 (Prediction + Visualization)

→ 이미지 넣고 모델이 박스를 치는 결과를 matplotlib으로 그려보기